In [154]:
import pandas as pd

In [155]:
data = pd.read_csv('../data/predicted_sentiment.csv') 

In [156]:
data.columns

Index(['product', 'categories', 'doRecommend', 'rating', 'feedback',
       'sentiment', 'predicted_sentiment'],
      dtype='object')

In [157]:
summary = data.groupby(['product', 'categories']).agg({
    'predicted_sentiment': 'mean',
    'rating': ['mean', 'count'],
    'doRecommend': 'mean',
    'feedback': 'first' 
}).reset_index()

In [158]:
summary.columns = [
    'product', 'categories',
    'avg_sentiment',
    'avg_rating', 'rating_count',
    'avg_doRecommend', 'sample_feedback'
]

In [159]:
summary['product'] = summary['product'].apply(lambda x: x.title() if isinstance(x, str) else x)

In [160]:
summary['rank'] = summary['avg_sentiment']*summary['rating_count']*summary['avg_rating']

In [161]:
import ast
summary['categories'] = summary['categories'].apply(ast.literal_eval)

In [162]:
summary.reset_index(drop=True, inplace=True)
summary.index.name = 'summary_index'

In [163]:
summary.to_csv('../data/summary.csv', index=True)

In [164]:
summary = summary.explode('categories', ignore_index=False)
summary.dropna(subset=['categories'], inplace=True)
summary = summary[summary['categories'] != ""]

In [165]:
summary.drop_duplicates(subset=['product'], inplace=True)

In [166]:
top3_per_category = (
    summary
    .sort_values(['categories', 'rank'], ascending=[True, False])
    .groupby('categories')
    .head(3)
)

top3_per_category = (
    top3_per_category
    .groupby(['product', 'avg_sentiment', 'avg_rating', 'rating_count', 'avg_doRecommend', 'sample_feedback', 'rank'], as_index=False)
    .agg({'categories': lambda x: list(x)})
)

top3_per_category.sort_values('rank', ascending=False, inplace=True)

In [167]:
top3_per_category.to_csv('../data/output_best_selling.csv', index=False)

In [168]:
# Least selling products: sort by 'rating_count' ascending
least_selling = (
    summary
    .sort_values('rating_count', ascending=True)
    .groupby('categories')
    .drop_duplicates('product')
    .head(3)
    )

least_selling = (
    least_selling
    .groupby(['product', 'avg_sentiment', 'avg_rating', 'rating_count', 'avg_doRecommend', 'sample_feedback', 'rank'], as_index=False)
    .agg({'categories': lambda x: list(x)})
)

least_selling.sort_values('rank', ascending=True, inplace=True)


AttributeError: 'DataFrameGroupBy' object has no attribute 'drop_duplicates'

In [ ]:
least_selling.to_csv('../data/output_least_selling.csv', index=False)